1. Distinct Monthly Orders (Time Series Forecasting)

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Group data to calculate monthly distinct orders
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d.%m.%Y')
data['Month'] = data['Order Date'].dt.to_period('M')
monthly_orders = data.groupby('Month')['Customer Order Code'].nunique().reset_index()

# Prepare data for SARIMA
monthly_orders.set_index('Month', inplace=True)
sarima_model = SARIMAX(monthly_orders, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_fit = sarima_model.fit(disp=False)
forecast = sarima_fit.get_forecast(steps=5).predicted_mean


2. Classification Model (Choice Model)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Encode categorical features
data['Season'] = data['Order Date'].dt.month % 12 // 3 + 1  # Map months to seasons
encoder = OneHotEncoder(sparse=False)
season_encoded = encoder.fit_transform(data[['Season']])

# Prepare features and target
X = np.hstack((season_encoded, pd.get_dummies(data[['Route', 'Customer Country Code']])))
y = data['Product Code']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
classifier.fit(X_train, y_train)

# Evaluate model
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))


3. Quantity Demanded (Regression)

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_absolute_error
from scipy.stats import mstats

# Empirical quantiles for quantity
quantiles = mstats.mquantiles(data['Items'], prob=[0.25, 0.5, 0.75])

# Features and target
X = pd.get_dummies(data[['Customer Country Code', 'Route', 'Value']])
y = data['Items']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Lasso model
lasso = LassoCV(cv=5).fit(X_train, y_train)

# Evaluate model
y_pred = lasso.predict(X_test)
print(mean_absolute_error(y_test, y_pred))


4. Demand Lead Time

In [ ]:
# Calculate lead time
data['Requested Delivery Date'] = pd.to_datetime(data['Requested Delivery Date'], format='%d.%m.%Y')
data['Lead Time'] = (data['Requested Delivery Date'] - data['Order Date']).dt.days

# Empirical quantiles for lead time
lead_time_quantiles = mstats.mquantiles(data['Lead Time'], prob=[0.25, 0.5, 0.75])


5. Monte Carlo Simulation

In [ ]:
# Monte Carlo Simulation
simulated_demand = []
for _ in range(1000):  # Run 1000 simulations
    orders = np.random.choice(monthly_orders['Customer Order Code'], size=12, replace=True)
    quantities = np.random.normal(loc=quantiles[1], scale=quantiles[2]-quantiles[0], size=12)
    lead_times = np.random.normal(loc=lead_time_quantiles[1], scale=lead_time_quantiles[2]-lead_time_quantiles[0], size=12)
    simulated_demand.append((orders, quantities, lead_times))


6. Advance vs. Urgent Demand

In [ ]:
# Distinguish advance vs. urgent demand
actual_advance_demand = test_data[test_data['Lead Time'] >= 40]  # Assuming threshold
simulated_advance_demand = [d for d in simulated_demand if d[2] >= 40]
